# Lost in the middle : 长上下文精度问题

In [11]:
from dotenv import load_dotenv
load_dotenv()
import os

# print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_API_BASE"))
#使用openai的官方sdk

https://openai.api2d.net


In [ ]:
from langchain.chains import LLMChain,StuffDocumentsChain
from langchain.document_transformers import (
    LongContextReorder
)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import  Chroma

# from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]
 
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
# print(embeddings)

#使用huggingface托管的开源LLM来做嵌入，MiniLM-L6-v2是一个较小的模型，可以在上运行
embedings = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2")

text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登之环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里.伯德是一位标志性的NBA球员。"
]
retrieval = Chroma.from_texts(text,embedings).as_retriever(
    search_kwargs={"k": 10}
)
query = "关于我的喜好都知道什么?"

#根据相关性返回文本块
docs = retrieval.get_relevant_documents(query)
docs


In [ ]:
#对检索结果进行排序
#问题相关性越低的内容块放在中间
#问题相关性越高的内容块放在头尾

reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs)

#头尾共有4个高相关性内容块
reo_docs

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")

#检查这种方案的精度效果

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

#设置llm
llm = OpenAI(api_key=api_key, api_base=api_base,model="gpt-3.5-turbo",temperature=0)

#设置prompt模板
docs_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}",
)
stuff_prompt_override = """Given this text extracts:
----------------------------------
{content}
----------------------------------
Please extract the most relevant information from the text.
{query}
"""
prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["content", "query"],
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

WorkChain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=docs_prompt,
    document_variable_name="content",
)

WorkChain.run(
    input_documents=reo_docs,query="关于我的喜好都知道什么?"
)